In [1]:
import polars as pl
import xgboost as xgb
model_path = "model_output/selected_features_xgb/one_model/features_v1_with_all/model_par_2/top120/xgb_ranker_model.bin"
parquet_path = "data/train_filled_v1_with_all.parquet"
xgb_model = xgb.Booster(model_file=model_path)
# 確認 feature_names
model_features = xgb_model.feature_names
if model_features is None:
    raise ValueError("❌ 模型沒有 feature_names，請確認訓練時有指定 feature_names")
print(f"✅ 模型共 {len(model_features)} 個features")

# 讀取 test_filled
df = pl.read_parquet(parquet_path)
print(f"✅ 讀取 test_filled，共 {df.height} rows")

# 檢查缺失
missing_in_data = [f for f in model_features if f not in df.columns]
if missing_in_data:
    raise ValueError(f"❌ 下列特徵在 test_filled 不存在: {missing_in_data}")

model_features = model_features + ["Id", "ranker_id"]
# 篩選&排序
df = df.select(model_features)


✅ 模型共 120 個features
✅ 讀取 test_filled，共 18145372 rows


In [2]:
import os
from scripts.recall import predict_and_get_topk

result_df = predict_and_get_topk(xgb_model, df, topk=10)

# 假設你要存的路徑
output_dir = "data/recall/top30/"
output_file = "train_all_top120.parquet"

# 建立資料夾
os.makedirs(output_dir, exist_ok=True)

# 寫入 parquet
save_path = os.path.join(output_dir, output_file)
result_df.write_parquet(save_path)

print(f"✅ 已儲存結果到 {save_path}")


✅ Top10 數量: 977451
✅ 已儲存結果到 data/recall/top30/train_all_top120.parquet


# umap

In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import umap
import joblib
import gc

# 讀取 top features
df = pd.read_csv("model_output/all_features_xgb/v1_add_all/v1_model/model_importance/common_features_with_ranks.csv")
filtered_features = df[df["min_rank_overall"] != 9999]["feature"].tolist()

# ========== 讀 recall結果 ==========
train_recall = pl.read_parquet("data/recall/top30/train_all_top120.parquet")
test_recall = pl.read_parquet("data/recall/top30/test_all_top120.parquet")

# 取top120 == True的Id
# 拿第2個欄名
second_col_name = train_recall.columns[1]

train_ids = train_recall.filter(pl.col(second_col_name) == True)["Id"].to_list()
test_ids = test_recall.filter(pl.col(test_recall.columns[1]) == True)["Id"].to_list()

print(f"✅ Train recall數量: {len(train_ids)}")
print(f"✅ Test recall數量: {len(test_ids)}")

# ========== 讀取原始 parquet ==========
train_filled = pl.read_parquet("data/train_filled_v1_with_all.parquet")
test_filled = pl.read_parquet("data/test_filled_v1_with_all.parquet")

# 需要保留的欄位
cols_to_keep = filtered_features + ["ranker_id", "companyID", "Id"]

# 篩選
train_filled = train_filled.select(cols_to_keep)
test_filled = test_filled.select(cols_to_keep)

# 過濾 Id
train_sample = train_filled.filter(pl.col("Id").is_in(train_ids))
test_sample = test_filled.filter(pl.col("Id").is_in(test_ids))

del train_filled, test_filled
gc.collect()

# 加來源標籤
train_sample = train_sample.with_columns(pl.lit("train").alias("source"))
test_sample = test_sample.with_columns(pl.lit("test").alias("source"))

# 確認共同欄位
common_cols = [c for c in train_sample.columns if c in test_sample.columns]
print(f"✅ 共有欄位: {common_cols}")

# 統一型別
def unify_dtypes(df):
    return df.with_columns([
        (
            pl.col(c).cast(pl.Float32)
            if df.schema[c] in pl.NUMERIC_DTYPES
            else pl.col(c).cast(pl.Utf8)
        )
        for c in df.columns
    ])

train_sample = unify_dtypes(train_sample)
test_sample = unify_dtypes(test_sample)

# 再concat
sample_all = pl.concat([train_sample, test_sample])

del train_sample, test_sample
gc.collect()

print(f"✅ 合併後 shape: {sample_all.shape}")

n_per_group = 5

# Polars ➡️ Pandas
df_pd = sample_all.to_pandas()

# groupby.sample
df_pd_sampled = (
    df_pd.groupby("ranker_id", group_keys=False, sort=False)
    .apply(lambda g: g.sample(n=min(n_per_group, len(g)), random_state=42))
    .reset_index(drop=True)
)

# 回Polars
sample_all = pl.from_pandas(df_pd_sampled)

# 檢查抽樣後數量
print(f"✅ 抽樣後總筆數: {sample_all.shape[0]}")

# 去掉id/metadata欄
X_numeric = (
    sample_all
    .drop([c for c in ["Id", "companyID", "ranker_id", "source"] if c in sample_all.columns])
    .to_numpy()
)

source_labels = sample_all["source"].to_list()
print(f"✅ UMAP輸入 shape: {X_numeric.shape}")


/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Train recall數量: 977451
✅ Test recall數量: 1019273
✅ 共有欄位: ['total_num_transfers_rank', 'legs0_segments0_flightNumber', 'price_per_duration_rank', 'price_percentile', 'price_from_median_zscore', 'price_per_duration', 'legs0_segments0_baggageAllowance_quantity', 'price_minus_fee_rank', 'legs0_arrivalAt_hour', 'pricingInfo_isAccessTP', 'legs0_departureAt_hour', 'both_legs_carrier_all_same', 'total_weighted_mean_cabin', 'price_per_fee_rank', 'totalPrice_rank', 'days_before_departure', 'legs0_segments0_seatsAvailable', 'price_per_fee', 'price_per_tax', 'legs1_main_carrier', 'isVip', 'leg0_view_diff_mean', 'legs1_departureAt_hour', 'baggage_total', 'duration_ratio', 'total_fees', 'all_view_diff_mean', 'legs1_arrivalAt_hour', 'log_price', 'legs1_segments0_flightNumber', 'legs1_weighted_mean_cabin', 'companyID_loo_mean_legs0_departureAt_hour', 'legs0_segments0_cabinClass', 'price_minus_fee', 'legs0_main_carrier', 'companyID_loo_mean_legs0_arrivalAt_hour', 'baggage_total_rank', 'legs0_max_durat

/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_40555/2435156377.py:57: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  if df.schema[c] in pl.NUMERIC_DTYPES


✅ 合併後 shape: (1996724, 203)


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_40555/2435156377.py:82: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(n_per_group, len(g)), random_state=42))


✅ 抽樣後總筆數: 1387106
✅ UMAP輸入 shape: (1387106, 199)


In [ ]:
del sample_all, train_recall, test_recall, test_ids, train_ids, df, df_pd, df_pd_sampled
gc.collect()

0

: 

In [ ]:

# Fit UMAP
umap_model = umap.UMAP(n_components=10, random_state=42)
X_umap_embedded = umap_model.fit_transform(X_numeric)

# 儲存
joblib.dump(umap_model, "umap_model_rowlevel_recall_top120.joblib")
print("✅ UMAP模型已儲存")


/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


# select data

In [1]:
import polars as pl
import pandas as pd
import numpy as np
import umap
import joblib
import gc

# 讀取 top features
df = pd.read_csv("model_output/all_features_xgb/v1_add_all/v1_model/model_importance/common_features_with_ranks.csv")
filtered_features = df[df["min_rank_overall"] != 9999]["feature"].tolist()

# ========== 讀 recall結果 ==========
train_recall = pl.read_parquet("data/recall/top30/train_all_top120.parquet")
test_recall = pl.read_parquet("data/recall/top30/test_all_top120.parquet")

# 取top120 == True的Id
# 拿第2個欄名
second_col_name = train_recall.columns[1]

train_ids = train_recall.filter(pl.col(second_col_name) == True)["Id"].to_list()
test_ids = test_recall.filter(pl.col(test_recall.columns[1]) == True)["Id"].to_list()

print(f"✅ Train recall數量: {len(train_ids)}")
print(f"✅ Test recall數量: {len(test_ids)}")

# ========== 讀取原始 parquet ==========
train_filled = pl.read_parquet("data/train_filled_v1_with_all.parquet")
test_filled = pl.read_parquet("data/test_filled_v1_with_all.parquet")

# 需要保留的欄位
cols_to_keep = filtered_features + ["ranker_id", "companyID", "Id", "selected"]

# 篩選
train_filled = train_filled.select(cols_to_keep)
test_filled = test_filled.select([c for c in cols_to_keep if c in test_filled.columns])
# 過濾 Id
train_sample = train_filled.filter(pl.col("Id").is_in(train_ids))
test_sample = test_filled.filter(pl.col("Id").is_in(test_ids))

del train_filled, test_filled
gc.collect()


/opt/anaconda3/envs/FlightRank/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Train recall數量: 2542178
✅ Test recall數量: 1019273


26785

In [4]:
import os

# 指定要儲存的資料夾
save_dir = "data/recall/top30/data"

# 建立資料夾（若不存在）
os.makedirs(save_dir, exist_ok=True)

# 儲存 parquet
train_sample_path = os.path.join(save_dir, "train_all_data.parquet")
test_sample_path = os.path.join(save_dir, "test_all_data.parquet")

train_sample.write_parquet(train_sample_path)
test_sample.write_parquet(test_sample_path)

print(f"✅ Train sample 已儲存到: {train_sample_path}")
print(f"✅ Test sample 已儲存到: {test_sample_path}")


✅ Train sample 已儲存到: data/recall/top30/data/train_all_data.parquet
✅ Test sample 已儲存到: data/recall/top30/data/test_all_data.parquet


In [2]:
train_sample

total_num_transfers_rank,legs0_segments0_flightNumber,price_per_duration_rank,price_percentile,price_from_median_zscore,price_per_duration,legs0_segments0_baggageAllowance_quantity,price_minus_fee_rank,legs0_arrivalAt_hour,pricingInfo_isAccessTP,legs0_departureAt_hour,both_legs_carrier_all_same,total_weighted_mean_cabin,price_per_fee_rank,totalPrice_rank,days_before_departure,legs0_segments0_seatsAvailable,price_per_fee,price_per_tax,legs1_main_carrier,isVip,leg0_view_diff_mean,legs1_departureAt_hour,baggage_total,duration_ratio,total_fees,all_view_diff_mean,legs1_arrivalAt_hour,log_price,legs1_segments0_flightNumber,legs1_weighted_mean_cabin,companyID_loo_mean_legs0_departureAt_hour,legs0_segments0_cabinClass,price_minus_fee,legs0_main_carrier,companyID_loo_mean_legs0_arrivalAt_hour,baggage_total_rank,…,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration_rank,legs1_segments2_key_view_count_rank,legs1_num_transfers_rank,legs1_segments2_duration,legs1_segments1_operatingCarrier_code_in_ff,legs0_segments2_flightNumber,legs0_segments2_aircraft_code,legs1_segments1_marketingCarrier_code_in_ff,legs0_segments2_seatsAvailable,legs1_segments2_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_operatingCarrier_code_in_ff,legs0_segments2_key_view_count_rank,is_cheapest,legs0_segments2_marketingCarrier_code,legs0_segments2_operatingCarrier_code_in_ff,legs0_segments2_duration,legs1_segments2_arrivalTo_airport_iata,legs1_segments2_aircraft_code,legs0_segments2_duration_rank,legs1_segments2_seatsAvailable,legs0_segments2_baggageAllowance_quantity,legs0_segments2_arrivalTo_airport_iata,legs0_segments2_arrivalTo_airport_city_iata,legs1_segments2_flightNumber,legs0_segments2_departureFrom_airport_iata,legs0_segments2_key_view_count,legs0_segments2_operatingCarrier_code,legs1_segments2_marketingCarrier_code,legs1_segments2_key_view_count,legs0_segments2_marketingCarrier_code_in_ff,ranker_id,companyID,Id,selected
u32,i32,u32,f64,f64,f64,f64,i32,i8,f64,i8,i8,f64,i32,u32,i32,f64,f64,f64,i32,i8,f64,i8,f64,f64,f64,f64,i8,f64,i32,f64,f64,i64,f64,i32,f64,i32,…,i32,i32,u32,u32,i64,i8,i32,i32,i8,f64,f64,f64,i64,i8,u32,i8,i32,i8,i64,i32,i32,i32,f64,f64,i32,i32,i32,i32,u32,i32,i32,u32,i8,str,i64,i64,i64
1,1274,9,0.04,-1.947024,53.43038,1.0,5,16,1.0,15,1,1.0,3,1,29,9.0,16884.0,45.509434,130,0,-71.04,9,2.0,1.025641,0.0,-22.08,14,9.734181,1263,1.0,12.454769,1,16884.0,130,12.44708,1,…,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,1,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""98ce0dabf6964640b63079fbafd42c…",57323,0,1
2,4360,10,0.18,-0.135934,53.759201,1.0,4,14,1.0,9,0,1.0,5,2,29,4.0,8.813136,22.813476,177,0,2.96,22,2.0,0.879447,5800.0,0.92,8,10.842048,4338,1.0,12.454769,1,45325.0,177,12.44708,1,…,678,1,1,2,0,1,8790,118,1,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""98ce0dabf6964640b63079fbafd42c…",57323,1,0
2,4360,11,0.42,0.0,56.461619,1.0,3,14,1.0,9,0,1.0,4,3,29,4.0,9.256163,23.960286,177,0,2.96,22,2.0,0.879447,5800.0,0.92,8,10.891094,4338,1.0,12.454769,1,47895.0,177,12.44708,1,…,678,1,1,2,0,1,8790,118,1,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""98ce0dabf6964640b63079fbafd42c…",57323,2,0
2,4360,20,0.66,1.490774,86.098843,1.0,2,14,1.0,9,0,1.0,2,4,29,4.0,81880.0,36.53726,177,0,2.96,22,2.0,0.879447,0.0,0.92,8,11.313022,4338,1.0,12.454769,1,81880.0,177,12.44708,1,…,678,1,1,2,0,1,8790,118,1,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""98ce0dabf6964640b63079fbafd42c…",57323,3,0
2,4360,21,0.9,1.712393,90.504732,1.0,1,14,1.0,9,0,1.0,1,5,29,4.0,86070.0,38.406961,177,0,2.96,22,2.0,0.879447,0.0,0.92,8,11.362928,4338,1.0,12.454769,1,86070.0,177,12.44708,1,…,678,1,1,2,0,1,8790,118,1,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""98ce0dabf6964640b63079fbafd42c…",57323,4,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

In [3]:
test_sample

total_num_transfers_rank,legs0_segments0_flightNumber,price_per_duration_rank,price_percentile,price_from_median_zscore,price_per_duration,legs0_segments0_baggageAllowance_quantity,price_minus_fee_rank,legs0_arrivalAt_hour,pricingInfo_isAccessTP,legs0_departureAt_hour,both_legs_carrier_all_same,total_weighted_mean_cabin,price_per_fee_rank,totalPrice_rank,days_before_departure,legs0_segments0_seatsAvailable,price_per_fee,price_per_tax,legs1_main_carrier,isVip,leg0_view_diff_mean,legs1_departureAt_hour,baggage_total,duration_ratio,total_fees,all_view_diff_mean,legs1_arrivalAt_hour,log_price,legs1_segments0_flightNumber,legs1_weighted_mean_cabin,companyID_loo_mean_legs0_departureAt_hour,legs0_segments0_cabinClass,price_minus_fee,legs0_main_carrier,companyID_loo_mean_legs0_arrivalAt_hour,baggage_total_rank,…,companyID_mode_has_transfer,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration_rank,legs1_segments2_key_view_count_rank,legs1_num_transfers_rank,legs1_segments2_duration,legs1_segments1_operatingCarrier_code_in_ff,legs0_segments2_flightNumber,legs0_segments2_aircraft_code,legs1_segments1_marketingCarrier_code_in_ff,legs0_segments2_seatsAvailable,legs1_segments2_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_operatingCarrier_code_in_ff,legs0_segments2_key_view_count_rank,is_cheapest,legs0_segments2_marketingCarrier_code,legs0_segments2_operatingCarrier_code_in_ff,legs0_segments2_duration,legs1_segments2_arrivalTo_airport_iata,legs1_segments2_aircraft_code,legs0_segments2_duration_rank,legs1_segments2_seatsAvailable,legs0_segments2_baggageAllowance_quantity,legs0_segments2_arrivalTo_airport_iata,legs0_segments2_arrivalTo_airport_city_iata,legs1_segments2_flightNumber,legs0_segments2_departureFrom_airport_iata,legs0_segments2_key_view_count,legs0_segments2_operatingCarrier_code,legs1_segments2_marketingCarrier_code,legs1_segments2_key_view_count,legs0_segments2_marketingCarrier_code_in_ff,ranker_id,companyID,Id
u32,i32,u32,f64,f64,f64,f64,i32,i8,f64,i8,i8,f64,i32,u32,i32,f64,f64,f64,i32,i8,f64,i8,f64,f64,f64,f64,i8,f64,i32,f64,f64,i64,f64,i32,f64,i32,…,i64,i32,i32,u32,u32,i64,i8,i32,i32,i8,f64,f64,f64,i64,i8,u32,i8,i32,i8,i64,i32,i32,i32,f64,f64,i32,i32,i32,i32,i64,i32,i32,i64,i8,str,i64,i64
1,456,31,0.212379,-0.224798,44.360759,1.0,46,11,1.0,6,1,1.0,48,11,50,4.0,2.502767,13.756624,184,0,94646.898058,21,2.0,0.903614,5600.0,87655.635922,21,9.548169,450,1.0,12.98,1,8418.0,184,12.94,2,…,0,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""c9373e5f772e43d593dd6ad2fa90f6…",62840,18144680
1,1867,45,0.256068,-0.185321,49.083056,1.0,29,12,1.0,8,1,1.0,28,14,50,7.0,9.842771,4.497412,197,0,-75921.101942,20,2.0,0.869565,1500.0,-68172.364078,21,9.600692,1838,1.0,12.98,1,13274.0,197,12.94,2,…,0,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""c9373e5f772e43d593dd6ad2fa90f6…",62840,18144694
1,456,44,0.29733,-0.175191,48.128617,1.0,44,11,1.0,6,1,1.0,47,16,50,4.0,2.67238,14.688911,184,0,94646.898058,18,2.0,0.931677,5600.0,87655.635922,19,9.613737,448,1.0,12.98,1,9368.0,184,12.94,2,…,0,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""c9373e5f772e43d593dd6ad2fa90f6…",62840,18144698
1,1867,54,0.317961,-0.127882,52.737542,1.0,26,12,1.0,8,1,1.0,26,17,50,7.0,10.575616,4.832268,197,0,-75921.101942,16,2.0,0.869565,1500.0,-68172.364078,16,9.672501,1822,1.0,12.98,1,14374.0,197,12.94,2,…,0,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0.0,0.0,678,678,8790,678,0,233,233,0,0,"""c9373e5f772e43d593dd6ad2fa90f6…",62840,18144720
1,1867,54,0.317961,-0.127882,52.737542,1.0,26,12,1.0,8,1,1.0,26,17,50,7.0,10.575616,4.832268,197,0,-75921.101942,19,2.0,0.869565,1500.0,-68172.364078,19,9.672501,1801,1.0,12.98,1,14374.0,197,12.94,2,…,0,678,1,1,1,0,0,8790,118,0,0.0,0.0,0.0,0,0,1,0,233,0,0,678,118,1,0